# Telco Churn Predictions

### Question: Why are customers churning? 

### Goals: Predict which customers are going to churn and what features are causing churn.
(Baseline of 60% churn***)

### Additional questons:
- Could the month in which they sign up influence churn?
- If a cohort(s) is identified by tenure, is there a cohort who has a higher rate of churn than others?
- Are there features that indicate a higher rate of churn?
- Is there a price threshhold for specific services where likihood of churn increases once customers get past that point? What is that price point and for what services?
- Are the rates comparable for customers who have month-to-month contracts at the 12-month mark vs the customers who have 1-year contracts at their 12-month mark?

## Imports

- Import all the neccessary libraries needed to create models that will predict churn.

In [35]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

from util import get_db_url
import acquire

- I will use the acquire.py file to get the data. I will look through the data in this notebook and clean up anything that I feel needs to be cleande up: fill or drop null values, change data types, etc.

**Tasks: check for nulls, look at dtypes, handle customer_id column.**

In [2]:
df = acquire.acquire_telco()
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,...,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,1,1,1,0015-UOCOJ,Female,1,No,No,7,Yes,...,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,1,1,1,0023-HGHWL,Male,1,No,No,1,No,...,No,No,No,Yes,25.1,25.10,Yes,Month-to-month,DSL,Electronic check
4,3,1,1,0032-PGELS,Female,0,Yes,Yes,1,No,...,No,No,No,No,30.5,30.50,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [3]:
df.shape

(7032, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 24 columns):
payment_type_id             7032 non-null int64
internet_service_type_id    7032 non-null int64
contract_type_id            7032 non-null int64
customer_id                 7032 non-null object
gender                      7032 non-null object
senior_citizen              7032 non-null int64
partner                     7032 non-null object
dependents                  7032 non-null object
tenure                      7032 non-null int64
phone_service               7032 non-null object
multiple_lines              7032 non-null object
online_security             7032 non-null object
online_backup               7032 non-null object
device_protection           7032 non-null object
tech_support                7032 non-null object
streaming_tv                7032 non-null object
streaming_movies            7032 non-null object
paperless_billing           7032 non-null object
monthly_charges 

**From looking at this table:**
- Change total_charges column into float.
- Change churn column into numeric values to run through a model.

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
payment_type_id,7032.0,2.315557,1.149523,1.00,1.0000,2.000,3.0000,4.00
internet_service_type_id,7032.0,1.872582,0.737271,1.00,1.0000,2.000,2.0000,3.00
contract_type_id,7032.0,1.688567,0.832934,1.00,1.0000,1.000,2.0000,3.00
senior_citizen,7032.0,0.162400,0.368844,0.00,0.0000,0.000,0.0000,1.00
tenure,7032.0,32.421786,24.545260,1.00,9.0000,29.000,55.0000,72.00
monthly_charges,7032.0,64.798208,30.085974,18.25,35.5875,70.350,89.8625,118.75
total_charges,7032.0,2283.300441,2266.771362,18.80,401.4500,1397.475,3794.7375,8684.80


- Check the data for any null values using .value_counts().

In [6]:
df.churn.value_counts()

No     5163
Yes    1869
Name: churn, dtype: int64

In [7]:
df.monthly_charges.value_counts()
df.monthly_charges.isnull().sum()

0

- Looking at the .value_counts() for total charges, I saw that there were 11 null values, so I went in and took care of those empty values by turning them into np.nan's, then double checked the work.

In [8]:
df.total_charges.value_counts()
df.total_charges.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df.total_charges.value_counts(dropna=False)

20.20      11
19.75       9
20.05       8
19.90       8
19.65       8
19.55       7
45.30       7
20.15       6
20.25       6
19.45       6
20.45       5
20.30       5
69.65       4
69.90       4
19.40       4
70.60       4
20.50       4
20.40       4
19.50       4
49.90       4
69.95       4
19.95       4
19.30       4
75.30       4
44.00       4
19.20       4
19.85       4
74.70       4
50.15       4
69.60       4
           ..
1834.15     1
999.80      1
8333.95     1
7171.70     1
3726.15     1
80.05       1
456.85      1
96.05       1
1399.35     1
2497.35     1
7069.30     1
688.65      1
6471.85     1
552.10      1
3242.50     1
3252.00     1
183.75      1
6886.25     1
7108.20     1
78.45       1
1171.30     1
3181.80     1
2540.10     1
3580.95     1
6470.10     1
3143.65     1
1110.50     1
1098.85     1
156.10      1
30.50       1
Name: total_charges, Length: 6530, dtype: int64

- Since there were only 11 np.nan values in the total charges, I opted to just drop the 11 rows from the dataframe, since I am working with over 7,000 rows.

In [9]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 24 columns):
payment_type_id             7032 non-null int64
internet_service_type_id    7032 non-null int64
contract_type_id            7032 non-null int64
customer_id                 7032 non-null object
gender                      7032 non-null object
senior_citizen              7032 non-null int64
partner                     7032 non-null object
dependents                  7032 non-null object
tenure                      7032 non-null int64
phone_service               7032 non-null object
multiple_lines              7032 non-null object
online_security             7032 non-null object
online_backup               7032 non-null object
device_protection           7032 non-null object
tech_support                7032 non-null object
streaming_tv                7032 non-null object
streaming_movies            7032 non-null object
paperless_billing           7032 non-null object
monthly_charges 

- I will now change the total_charges from an object to a float, since it is money.

In [10]:
df.total_charges.dtype
df['total_charges'] = df['total_charges'].astype(float)
df.total_charges.dtype

dtype('float64')

**Task: Look into customer_id column.**

## Baseline Model: Decision Tree

- After preparing the data, I will create a new baseline dataframe and pick random numeric features to create a baseline model to predict churn. In order to do this, I need need to encode my churn column into 0's and 1's to depict either churned or not churned.

**Task: Encode churn column.**

In [11]:
df_base = df
df_base.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,...,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,1,1,1,0015-UOCOJ,Female,1,No,No,7,Yes,...,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,1,1,1,0023-HGHWL,Male,1,No,No,1,No,...,No,No,No,Yes,25.1,25.10,Yes,Month-to-month,DSL,Electronic check
4,3,1,1,0032-PGELS,Female,0,Yes,Yes,1,No,...,No,No,No,No,30.5,30.50,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [12]:
base_encoder = LabelEncoder()
df_base.churn = base_encoder.fit_transform(df_base.churn)
df_base.churn.head()

0    0
1    0
2    0
3    1
4    1
Name: churn, dtype: int64

- The dtype for the churn column has now been changed from object to an integer.

In [13]:
df_base.churn.dtype

dtype('int64')

- I will define what columns I want to be my features (X) and churn as my target (y).

In [14]:
X = df_base[['tenure', 'monthly_charges', 'total_charges']]
y = df_base[['churn']]

- I will split the data into train and test, and then run the data through a decision tree model to create a baseline model.
- After I plug the data into the model, I will look at the predicted churn values, and the probabilities of the customer either churning or not churning.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=123)

In [16]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=123)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=123, splitter='best')

In [17]:
y_pred = clf.predict(X_train)
y_pred

array([1, 0, 0, ..., 0, 0, 1])

In [18]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.33333333, 0.66666667],
       [1.        , 0.        ],
       [0.88701923, 0.11298077],
       ...,
       [0.88701923, 0.11298077],
       [0.62608696, 0.37391304],
       [0.40869565, 0.59130435]])

#### Evaluate the Baseline Model

In [19]:
format(clf.score(X_train, y_train))

'0.8445753758634701'

In [20]:
confusion_matrix(y_train, y_pred)

array([[3354,  271],
       [ 494,  803]])

In [21]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.93      0.90      3625
           1       0.75      0.62      0.68      1297

    accuracy                           0.84      4922
   macro avg       0.81      0.77      0.79      4922
weighted avg       0.84      0.84      0.84      4922



### Takeaways:
- The accuracy score for the data chosen to run through this model is 84%. This baseline is better than the firt baseline, 60%.

**Tasks: Create a new model with more/different features, encoded data and scaled data.**

## Model 2: Logistic Regression 
(maybe)

- I will create a new dataframe with the cleaned up data and then encode the dtypes that are objects to turn them into numeric values to have more features to run through a model. I will set the customer id column as the index, to move it to the side, but keep it for reference.

**Task: Encode features run through a new model.**

In [22]:
df_telco = df
df_telco.set_index('customer_id', inplace=True)
df_telco.head()

,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
customer_id,,,,,,,,,,,,,,,,,,,,,
0003-MKNFE,2,1,1,Male,0,No,No,9,Yes,Yes,...,No,No,Yes,No,59.9,542.40,0,Month-to-month,DSL,Mailed check
0013-MHZWF,4,1,1,Female,0,No,Yes,9,Yes,No,...,Yes,Yes,Yes,Yes,69.4,571.45,0,Month-to-month,DSL,Credit card (automatic)
0015-UOCOJ,1,1,1,Female,1,No,No,7,Yes,No,...,No,No,No,Yes,48.2,340.35,0,Month-to-month,DSL,Electronic check
0023-HGHWL,1,1,1,Male,1,No,No,1,No,No phone service,...,No,No,No,Yes,25.1,25.10,1,Month-to-month,DSL,Electronic check
0032-PGELS,3,1,1,Female,0,Yes,Yes,1,No,No phone service,...,No,No,No,No,30.5,30.50,1,Month-to-month,DSL,Bank transfer (automatic)


- I will encode all columns that make sense for me to change with a Label Encoder in order to run more features through a model.
- I will also drop the columns contract typem internet service type and payment type, since they are repeats of the type id columns.

**Task: Establish what each number corresponds to after encoding the columns.**

In [23]:
def encode(df):
    encoder = LabelEncoder()
    df = df.drop(columns=['contract_type', 'internet_service_type','payment_type'])
    encode_list = ['gender','partner', 'dependents', 'phone_service','multiple_lines', 'online_security', 'online_backup','device_protection','tech_support','streaming_tv', \
                  'streaming_movies', 'paperless_billing', 'churn']
    for c in encode_list:
        df[c] = encoder.fit_transform(df[c])
    return df

In [24]:
df_encoded = encode(df_telco)
df_encoded.head()

,payment_type_id,internet_service_type_id,contract_type_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn
customer_id,,,,,,,,,,,,,,,,,,,,
0003-MKNFE,2,1,1,1,0,0,0,9,1,2,0,0,0,0,0,2,0,59.9,542.40,0
0013-MHZWF,4,1,1,0,0,0,1,9,1,0,0,0,0,2,2,2,1,69.4,571.45,0
0015-UOCOJ,1,1,1,0,1,0,0,7,1,0,2,0,0,0,0,0,1,48.2,340.35,0
0023-HGHWL,1,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,1,25.1,25.10,1
0032-PGELS,3,1,1,0,0,1,1,1,0,1,2,0,0,0,0,0,0,30.5,30.50,1


In [25]:
X = df_encoded.drop(['churn'],axis=1)
y = df_encoded[['churn']]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7, random_state=123)
y_train.head()

,churn
customer_id,
3866-MDTUB,1
5555-RNPGT,0
3727-OWVYD,0
2270-CHBFN,0
6892-XPFPU,1


In [37]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')
logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [38]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[-0.00303582 -0.00158883 -0.00244271 -0.00035774  0.00032112 -0.00046509
  -0.00060503 -0.02825036 -0.00060317  0.0004257  -0.00179617 -0.00102356
  -0.00101433 -0.0017803  -0.00028873 -0.00032554  0.00029718  0.01494817
  -0.00016739]]
Intercept: 
 [-0.0006933]


In [39]:
y_pred = logit.predict(X_train)
y_pred

array([1, 1, 0, ..., 0, 0, 1])

In [40]:
y_pred_proba = logit.predict_proba(X_train)
y_pred_proba

array([[0.27615712, 0.72384288],
       [0.49406463, 0.50593537],
       [0.60686144, 0.39313856],
       ...,
       [0.5675117 , 0.4324883 ],
       [0.51617906, 0.48382094],
       [0.30644296, 0.69355704]])

In [41]:
format(logit.score(X_train, y_train))

'0.7013409183258837'

In [42]:
print(confusion_matrix(y_train, y_pred))

[[2506 1119]
 [ 351  946]]


In [43]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.69      0.77      3625
           1       0.46      0.73      0.56      1297

    accuracy                           0.70      4922
   macro avg       0.67      0.71      0.67      4922
weighted avg       0.77      0.70      0.72      4922



In [44]:
print('Accuracy of Logistic Regression classifier on test set: {:.2f}'
     .format(logit.score(X_test, y_test)))

Accuracy of Logistic Regression classifier on test set: 0.70
